# Note !

**It required higher resource to train different models and see the effect of each model, so we have used paid server from vast.ai to just display some results and it can be saw in *Compare ML Models* notebook after training.**

So I just run some cells from the notebook, But the models we trained on all data are saved in **models direction".

In [ ]:
# Main libraries 
from sklearn.ensemble import BaggingClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import ExtraTreesClassifier 
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
from xgboost import XGBClassifier
import matplotlib.pyplot as plt

# Our files
from configs import *
from fetch_data import *
from features_extraction import *
from data_shuffling_split import *
from data_preprocess import *
from ml_modeling import *

In [ ]:
strat_train_set = read_csv("train/strat_train_set.csv")
strat_train_set.head()

In [ ]:
x_train_text, x_val_text, y_train, y_val = prepare_data(strat_train_set)

In [ ]:
x_train_text_tokenized = tokenize_using_nltk_TreebankWordTokenizer(x_train_text)

print("Before Tokenization : \n", x_train_text[:3])
print("="*50)
print("After Tokenization : \n", x_train_text_tokenized[:3])
print("="*50)

x_val_text_tokenized = tokenize_using_nltk_TreebankWordTokenizer(x_val_text)

print("Before Tokenization : \n", x_val_text[:3])
print("="*50)
print("After Tokenization : \n", x_val_text_tokenized[:3])

# Curse of Dimensional & sparsity

Tasks like **Computer Vision** or **Natural Language Processing** run to problem called **Curse of Dimensional**, and as we have here NLP classification problem, the number of instance are semi-large, but this not the point, the point is what we dealing with is text language, and the language are free of grammar, ritch of vocabulary and others.

So to handle like these problems we need to extract features from the text, the old or classical way is using BOW (Bag of Words), and this approach run to the problem of **Curse of Dimensionality** as we will have number of features related to the unique words in our data. Not just that most of these features are zeros, what is we called sparse matrix.

Beside of that, this matrix we will get from that approach represent the text not the word itself, so there is no similarity between words and other problem.

# Word2Vec

From what we have of these problem we moved to another approach related to the word representation.

Word2vec is numerical representation of dense vector for the word semantics of meaning, including the implies meaning of the word. So we can use these word representation in our text as we will see.

But to train Word2Vec and got a pretty good result of word representation, it first require massive data millions of text document, and second to wait for a while for your model to train. So we use the idea of transfer learning, and use some of the pre trained Arabic Word2Vec models and download it to use in our task. 

**Check for more information about the models we used:** [AraVec](https://www.sciencedirect.com/science/article/pii/S1877050917321749)



# Build Matrix of Text

Any ML or DL model require specific number of features (input) to dealing with, but what we have here with word2vec is word representation. So how it works for text ?

We will build a matrix for each text, but we need to limit the number of words in each text, because we can not train the model with different number of words in text.

# Note !

We can take the maximum number of words in the longest text, but maybe for some documents its has thousand of words, so we use the graph below and other helpful method to get reasonable length.

In [ ]:
# Get how many words inside each text after tokenization
num_of_words_in_each_text = [len(text) for text in x_train_text_tokenized]
max_len = max(num_of_words_in_each_text)
print("The max length is: ", max_len)
num_of_words_in_each_text[:10]

In [ ]:
# count how many times these value repeated and sort them
new_dicts = get_keys_that_val_gr_than_num(num_of_words_in_each_text, 1000)
keys = list(new_dicts.keys())
values = list(new_dicts.values())
plt.style.use('dark_background')
fig = plt.gcf()
fig.set_size_inches(18.5, 6)
plt.bar(range(len(new_dicts)), values, tick_label=keys)
plt.show()

In [ ]:
word_to_vec_model = load_word2vec_model("models/word2vec/bakrianoo_unigram_cbow_model/full_uni_cbow_100_twitter.mdl")

In [ ]:
number_of_features = 100
max_len_str = 64
word2vec_path = "bakr/"
model_path_to_save = "models/ml_models/"
estimators = voting_models()

X_train_embed_matrix = text_to_matrix_using_word2vec(word_to_vec_model, x_train_text_tokenized, max_len_str)
X_val_embed_matrix = text_to_matrix_using_word2vec(word_to_vec_model, x_val_text_tokenized, max_len_str)

# Train Logistic Regression

In [ ]:
model = LogisticRegression(penalty='l2', C=1, multi_class='multinomial', solver='lbfgs', verbose=1)
model = ml_classifer_pipeline(model, X_train_embed_matrix, y_train, X_val_embed_matrix, y_val,word2vec_path, model_path_to_save)

# Train SVC

In [ ]:
model = LinearSVC(C=0.5,  max_iter=50, verbose=1)
model = ml_classifer_pipeline(model, X_train_embed_matrix, y_train, X_val_embed_matrix, y_val,word2vec_path, model_path_to_save)

# Hard Voting

In [ ]:
model = VotingClassifier(estimators, voting="hard")
model = ml_classifer_pipeline(model, X_train_embed_matrix, y_train, X_val_embed_matrix, y_val,word2vec_path, model_path_to_save)

# Extr Tree Classifier

In [ ]:
model = ExtraTreesClassifier(n_estimators=100, max_depth=5, max_samples=.1, verbose=1)
model = ml_classifer_pipeline(model, X_train_embed_matrix, y_train, X_val_embed_matrix, y_val,word2vec_path, model_path_to_save)

# AdaBoost 

In [ ]:
model = LinearSVC(C=0.5,  verbose=1)
model = AdaBoostClassifier(model,  algorithm="SAMME", n_estimators=5)
model = ml_classifer_pipeline(model, X_train_embed_matrix, y_train, X_val_embed_matrix, y_val,word2vec_path, model_path_to_save)

#  Gradient Boosting

In [ ]:
model = GradientBoostingClassifier(n_estimators=10, subsample=.1, learning_rate=.5,   max_depth=5, verbose=1)
model = ml_classifer_pipeline(model, X_train_embed_matrix, y_train, X_val_embed_matrix, y_val,word2vec_path, model_path_to_save)

# XGBClassifier

In [ ]:
model = XGBClassifier(max_depth=5, subsample=.1)
model = ml_classifer_pipeline(model, X_train_embed_matrix, y_train, X_val_embed_matrix, y_val,word2vec_path, model_path_to_save)

# Rezk

In [ ]:
word_to_vec_model = load_word2vec_model("models/word2vec/rezk_unigram_CBOW_model/train_word2vec_cbow__window_3_min_count_300")

In [ ]:
number_of_features = 300
max_len_str = 64
word2vec_path = "rezk/"
model_path_to_save = "models/ml_models/"
estimators = voting_models()

X_train_embed_matrix = text_to_matrix_using_word2vec(word_to_vec_model, x_train_text_tokenized, max_len_str)
X_val_embed_matrix = text_to_matrix_using_word2vec(word_to_vec_model, x_val_text_tokenized, max_len_str)

# Train Logistic Regression

In [ ]:
model = LogisticRegression(penalty='l2', C=1, multi_class='multinomial', solver='lbfgs', verbose=1)
model = ml_classifer_pipeline(model, X_train_embed_matrix, y_train, X_val_embed_matrix, y_val,word2vec_path, model_path_to_save)

# Train SVC

In [ ]:
model = LinearSVC(C=0.5,  max_iter=50, verbose=1)
model = ml_classifer_pipeline(model, X_train_embed_matrix, y_train, X_val_embed_matrix, y_val,word2vec_path, model_path_to_save)

# Bagging Classifier

In [ ]:
dec_tree_cls = DecisionTreeClassifier(max_depth=5, min_samples_split=2, min_samples_leaf=1)
model = BaggingClassifier(base_estimator=dec_tree_cls, n_estimators=100, max_samples=.2, verbose=1)
ml_classifer_pipeline(model, X_train_embed_matrix, y_train, X_val_embed_matrix, y_val,word2vec_path, model_path_to_save)

# Extr Tree Classifier

In [ ]:
model = ExtraTreesClassifier(n_estimators=100, max_depth=5, max_samples=.1, verbose=1)
model = ml_classifer_pipeline(model, X_train_embed_matrix, y_train, X_val_embed_matrix, y_val,word2vec_path, model_path_to_save)

# AdaBoost 

In [ ]:
model = LinearSVC(C=0.5,  verbose=1)
model = AdaBoostClassifier(model,  algorithm="SAMME", n_estimators=5)
model = ml_classifer_pipeline(model, X_train_embed_matrix, y_train, X_val_embed_matrix, y_val,word2vec_path, model_path_to_save)

#  Gradient Boosting

In [ ]:
model = GradientBoostingClassifier(n_estimators=10, subsample=.1, learning_rate=.5,   max_depth=5, verbose=1)
model = ml_classifer_pipeline(model, X_train_embed_matrix, y_train, X_val_embed_matrix, y_val,word2vec_path, model_path_to_save)

# XGBClassifier

In [ ]:
model = XGBClassifier(max_depth=5, subsample=.1)
model = ml_classifer_pipeline(model, X_train_embed_matrix, y_train, X_val_embed_matrix, y_val,word2vec_path, model_path_to_save)

# Note !

**We can use different word embedding representation, to see its effect on training.**